# Aviation Accident Analysis

## Business Understanding
**Goal**: Identify lowest-risk aircraft for company expansion  
**Stakeholder**: Head of Aviation Division  
**Key Questions**:  
1. Which aircraft models have the fewest accidents?  
2. What factors correlate with safety?  

# SETUP

In [ ]:
# Import libraries

import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

In [6]:
pip install chardet

Note: you may need to restart the kernel to use updated packages.


In [14]:
df = pd.read_csv('AviationData.csv', encoding='latin1', on_bad_lines='skip')
df.head()

<ipython-input-14-401ef945b385>:1: DtypeWarning: Columns (6,7,28) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('AviationData.csv', encoding='latin1', on_bad_lines='skip')


,Event.Id,Investigation.Type,Accident.Number,Event.Date,Location,Country,Latitude,Longitude,Airport.Code,Airport.Name,...,Purpose.of.flight,Air.carrier,Total.Fatal.Injuries,Total.Serious.Injuries,Total.Minor.Injuries,Total.Uninjured,Weather.Condition,Broad.phase.of.flight,Report.Status,Publication.Date
0,20001218X45444,Accident,SEA87LA080,1948-10-24,"MOOSE CREEK, ID",United States,NaN,NaN,NaN,NaN,...,Personal,NaN,2.0,0.0,0.0,0.0,UNK,Cruise,Probable Cause,NaN
1,20001218X45447,Accident,LAX94LA336,1962-07-19,"BRIDGEPORT, CA",United States,NaN,NaN,NaN,NaN,...,Personal,NaN,4.0,0.0,0.0,0.0,UNK,Unknown,Probable Cause,19-09-1996
2,20061025X01555,Accident,NYC07LA005,1974-08-30,"Saltville, VA",United States,36.922223,-81.878056,NaN,NaN,...,Personal,NaN,3.0,NaN,NaN,NaN,IMC,Cruise,Probable Cause,26-02-2007
3,20001218X45448,Accident,LAX96LA321,1977-06-19,"EUREKA, CA",United States,NaN,NaN,NaN,NaN,...,Personal,NaN,2.0,0.0,0.0,0.0,IMC,Cruise,Probable Cause,12-09-2000
4,20041105X01764,Accident,CHI79FA064,1979-08-02,"Canton, OH",United States,NaN,NaN,NaN,NaN,...,Personal,NaN,1.0,2.0,NaN,0.0,VMC,Approach,Probable Cause,16-04-1980


In [15]:
print(df.shape)
df.head(3)

(88889, 31)


,Event.Id,Investigation.Type,Accident.Number,Event.Date,Location,Country,Latitude,Longitude,Airport.Code,Airport.Name,...,Purpose.of.flight,Air.carrier,Total.Fatal.Injuries,Total.Serious.Injuries,Total.Minor.Injuries,Total.Uninjured,Weather.Condition,Broad.phase.of.flight,Report.Status,Publication.Date
0,20001218X45444,Accident,SEA87LA080,1948-10-24,"MOOSE CREEK, ID",United States,NaN,NaN,NaN,NaN,...,Personal,NaN,2.0,0.0,0.0,0.0,UNK,Cruise,Probable Cause,NaN
1,20001218X45447,Accident,LAX94LA336,1962-07-19,"BRIDGEPORT, CA",United States,NaN,NaN,NaN,NaN,...,Personal,NaN,4.0,0.0,0.0,0.0,UNK,Unknown,Probable Cause,19-09-1996
2,20061025X01555,Accident,NYC07LA005,1974-08-30,"Saltville, VA",United States,36.922223,-81.878056,NaN,NaN,...,Personal,NaN,3.0,NaN,NaN,NaN,IMC,Cruise,Probable Cause,26-02-2007


# DATA CLEANING

In [17]:
print(df.columns.tolist())

['Event.Id', 'Investigation.Type', 'Accident.Number', 'Event.Date', 'Location', 'Country', 'Latitude', 'Longitude', 'Airport.Code', 'Airport.Name', 'Injury.Severity', 'Aircraft.damage', 'Aircraft.Category', 'Registration.Number', 'Make', 'Model', 'Amateur.Built', 'Number.of.Engines', 'Engine.Type', 'FAR.Description', 'Schedule', 'Purpose.of.flight', 'Air.carrier', 'Total.Fatal.Injuries', 'Total.Serious.Injuries', 'Total.Minor.Injuries', 'Total.Uninjured', 'Weather.Condition', 'Broad.phase.of.flight', 'Report.Status', 'Publication.Date']


In [18]:
# Select relevant columns
cols = ['Event.Date', 'Aircraft.Category', 'Make', 'Model', 
        'Injury.Severity', 'Total.Fatal.Injuries', 'Total.Serious.Injuries',
        'Broad.phase.of.flight', 'Weather.Condition']
df = df[cols].copy()

In [19]:
# Convert date
df['Event.Date'] = pd.to_datetime(df['Event.Date'], errors='coerce')


In [20]:
# Handle missing values
df['Aircraft.Category'] = df['Aircraft.Category'].fillna('Unknown')
df['Injury.Severity'] = df['Injury.Severity'].fillna('Unknown')

In [21]:
# Filter only airplanes
df = df[df['Aircraft.Category'].str.contains('Airplane', case=False, na=False)]

# ANALYSIS

In [22]:
# Extract year
df['Year'] = df['Event.Date'].dt.year

In [23]:
# Top 10 safest aircraft models
safest = df.groupby(['Make', 'Model']).agg(
    Total_Accidents=('Model', 'count'),
    Fatal_Accidents=('Total.Fatal.Injuries', lambda x: (x > 0).sum())
).sort_values('Fatal_Accidents').head(10)


In [24]:
# Accident trends by year
yearly_trends = df.groupby('Year').size()

In [26]:
 # Accident causes
phase_stats = df['Broad.phase.of.flight'].value_counts()
weather_stats = df['Weather.Condition'].value_counts()